In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
spark

In [3]:
bank_customer_data = spark.read.json("file:///home/hadoop/Downloads/bank_edited.json", multiLine=True)

In [4]:
bank_customer_data.show()

+---+-------+--------+-------+---+-------+--------+---------+-------+------------+----+--------+-----+-----+--------+--------+---+
|age|balance|campaign|contact|day|default|duration|education|housing|         job|loan| marital|month|pdays|poutcome|previous|  y|
+---+-------+--------+-------+---+-------+--------+---------+-------+------------+----+--------+-----+-----+--------+--------+---+
| 58|   2143|       1|unknown|  5|     no|     261| tertiary|    yes|  management|  no| married|  may|   -1| unknown|       0| no|
| 44|     29|       1|unknown|  5|     no|     151|secondary|    yes|  technician|  no|  single|  may|   -1| unknown|       0| no|
| 33|      2|       1|unknown|  5|     no|      76|secondary|    yes|entrepreneur| yes| married|  may|   -1| unknown|       0| no|
| 47|   1506|       1|unknown|  5|     no|      92|  unknown|    yes| blue-collar|  no| married|  may|   -1| unknown|       0| no|
| 33|      1|       1|unknown|  5|     no|     198|  unknown|     no|     unknown| 

In [5]:
bank_customer_data.printSchema()

root
 |-- age: long (nullable = true)
 |-- balance: long (nullable = true)
 |-- campaign: long (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: long (nullable = true)
 |-- default: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- education: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- job: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- month: string (nullable = true)
 |-- pdays: long (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- previous: long (nullable = true)
 |-- y: string (nullable = true)



#### 1. Display Max, Min and Mean Age of the targeted Customers

In [6]:
bank_customer_data.createOrReplaceTempView("banktable")

In [7]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [8]:
bank_customer_data.select(max("age")).show()
bank_customer_data.select(min("age")).show()
bank_customer_data.select(round(mean("age"))).show()

+--------+
|max(age)|
+--------+
|      95|
+--------+

+--------+
|min(age)|
+--------+
|      18|
+--------+

+------------------+
|round(avg(age), 0)|
+------------------+
|              41.0|
+------------------+



In [9]:
# spark sql - alternative method
spark.sql("select max(age), min(age), mean(age) from banktable").show()

+--------+--------+-----------------+
|max(age)|min(age)|         avg(age)|
+--------+--------+-----------------+
|      95|      18|40.93621021432837|
+--------+--------+-----------------+



#### 2. Check the quality of Customers by analyzing targeted customer's min, max, mean, median balance.

In [15]:
spark.sql("select min(abs(balance)), max(balance), mean(balance), \
          percentile_approx(balance, 0.5) as median_balance from banktable").show()

+-----------------+------------+------------------+--------------+
|min(abs(balance))|max(balance)|      avg(balance)|median_balance|
+-----------------+------------+------------------+--------------+
|                0|      102127|1362.2720576850766|           448|
+-----------------+------------+------------------+--------------+



#### 3. Check if age matters in marketing subscription for Term Deposit Scheme.

In [17]:
spark.sql("select age, count(*) as customer_count from banktable where y = 'yes' \
          group by age order by customer_count desc").show()

+---+--------------+
|age|customer_count|
+---+--------------+
| 32|           221|
| 30|           217|
| 33|           210|
| 35|           209|
| 31|           206|
| 34|           198|
| 36|           195|
| 29|           171|
| 37|           170|
| 28|           162|
| 38|           144|
| 39|           143|
| 27|           141|
| 26|           134|
| 41|           120|
| 46|           118|
| 40|           116|
| 25|           113|
| 47|           113|
| 42|           111|
+---+--------------+
only showing top 20 rows



#### 4. Calculate Marketing Success Rate.

In [24]:
spark.sql("select round((select count(*) from banktable where y = 'yes')*100/(count(*)), 3) \
as success_rate from banktable").show()

+------------+
|success_rate|
+------------+
|      11.698|
+------------+



In [26]:
spark.sql("select round((select count(*) from banktable where y = 'no')*100/(count(*)), 3) \
as failre_rate from banktable").show()

+-----------+
|failre_rate|
+-----------+
|     88.302|
+-----------+



In [27]:
spark.sql("""
            SELECT
                 COUNT(CASE WHEN y = 'yes' THEN 1 ELSE NULL END)*100/COUNT(*) as total_success_rate
            FROM
                banktable
                 """).show()

+------------------+
|total_success_rate|
+------------------+
|11.698480458295547|
+------------------+



#### 6. Check if marital status matters  in marketing subscription for term desposit scheme 

In [30]:
spark.sql("""
select marital, count(*) as marital_success from banktable where y = 'yes' group by marital
order by marital_success desc
""").show()

+--------+---------------+
| marital|marital_success|
+--------+---------------+
| married|           2755|
|  single|           1912|
|divorced|            622|
+--------+---------------+



#### 8. Check if age and marital status together mattered for subscription to term deposit scheme. 

In [33]:
spark.sql("""
select age, marital, count(*) as _success from banktable where y = 'yes' group by age, marital
order by _success desc
""").show()

+---+-------+--------+
|age|marital|_success|
+---+-------+--------+
| 30| single|     151|
| 28| single|     138|
| 29| single|     133|
| 32| single|     124|
| 26| single|     121|
| 34|married|     118|
| 31| single|     111|
| 27| single|     110|
| 35|married|     101|
| 36|married|     100|
| 25| single|      99|
| 37|married|      98|
| 33|married|      97|
| 33| single|      97|
| 32|married|      87|
| 39|married|      87|
| 38|married|      86|
| 35| single|      84|
| 47|married|      83|
| 31|married|      80|
+---+-------+--------+
only showing top 20 rows



#### 9. Compute Success rate of each age and marital status category.

In [40]:
#success rate of each age a
spark.sql("""
    select age, marital, count(*) as total_records,
        SUM(case when y = 'yes' then 1 else 0 end) AS count_sucess,
        (SUM(case when y = 'yes' then 1 else 0 end)*100/COUNT(*)) as success_percentage
    from banktable group by age, marital order by success_percentage desc
""").show()

+---+--------+-------------+------------+------------------+
|age| marital|total_records|count_sucess|success_percentage|
+---+--------+-------------+------------+------------------+
| 93| married|            2|           2|             100.0|
| 95|divorced|            1|           1|             100.0|
| 68|divorced|            6|           6|             100.0|
| 92| married|            2|           2|             100.0|
| 85|divorced|            1|           1|             100.0|
| 90|divorced|            2|           2|             100.0|
| 87|divorced|            1|           1|             100.0|
| 86|  single|            1|           1|             100.0|
| 67|divorced|            8|           7|              87.5|
| 62|divorced|            6|           5| 83.33333333333333|
| 76|divorced|            8|           6|              75.0|
| 85| married|            4|           3|              75.0|
| 71|divorced|           11|           8| 72.72727272727273|
| 87| married|          

#### 10. Do features engineering for Bank Investment Scheme and find Effect of Age on the Campaign.

In [45]:
spark.sql("""
select age_category, count(*) as success_count from (select case when age < 25 then 'Teenager'
            when age >= 25 and age <= 35 then 'Adult'
            when age > 35 and age <= 55 then 'Middle Aged'
            else 'old'
        end as age_category
        from banktable where y = 'yes') group by age_category 
order by success_count desc
""").show()

+------------+-------------+
|age_category|success_count|
+------------+-------------+
| Middle Aged|         2194|
|       Adult|         1982|
|         old|          906|
|    Teenager|          207|
+------------+-------------+



#### Alternative Method : Create UDF

In [47]:
from pyspark.sql.functions import udf

In [48]:
age_range = udf(lambda age : 'Teenager' if age < 25 else
                             'Adult' if (age >= 25 and age < 35) else
                             'Middle Aged' if (age >= 35 and age < 55) else
                             'Old'
                )

In [52]:
bank_customer_DF = bank_customer_data.withColumn('age_category', 
                                                 age_range(bank_customer_data.age))

In [53]:
bank_customer_DF.show()

+---+-------+--------+-------+---+-------+--------+---------+-------+------------+----+--------+-----+-----+--------+--------+---+------------+
|age|balance|campaign|contact|day|default|duration|education|housing|         job|loan| marital|month|pdays|poutcome|previous|  y|age_category|
+---+-------+--------+-------+---+-------+--------+---------+-------+------------+----+--------+-----+-----+--------+--------+---+------------+
| 58|   2143|       1|unknown|  5|     no|     261| tertiary|    yes|  management|  no| married|  may|   -1| unknown|       0| no|         Old|
| 44|     29|       1|unknown|  5|     no|     151|secondary|    yes|  technician|  no|  single|  may|   -1| unknown|       0| no| Middle Aged|
| 33|      2|       1|unknown|  5|     no|      76|secondary|    yes|entrepreneur| yes| married|  may|   -1| unknown|       0| no|       Adult|
| 47|   1506|       1|unknown|  5|     no|      92|  unknown|    yes| blue-collar|  no| married|  may|   -1| unknown|       0| no| Middl

In [60]:
bank_customer_DF.createOrReplaceTempView('newBankTable')

In [63]:
spark.sql("""
select age_category, count(*) as success_count from newBankTable group by age_category
order by success_count DESC
""").show()

+------------+-------------+
|age_category|success_count|
+------------+-------------+
| Middle Aged|        24492|
|       Adult|        14204|
|         Old|         5706|
|    Teenager|          809|
+------------+-------------+



In [55]:
spark.sql('show databases').show()

+------------+
|databaseName|
+------------+
|     default|
+------------+



#### 11. Write a Query to show distributed probability rate for each age category